In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os
from matplotlib.pyplot import hist
from scipy import stats
# import more functions or modules if you need them !!

In [5]:
# set random seed for numpy
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

In [6]:
oil_df = pd.read_pickle('../Data/data/comprehensive_new_full.pkl')

In [7]:
oil_df

,year,numcode,oilreserves_full,oilreserves,oilreserves_public,newdiscovery_aspo,aspo,wildcat,endowment,pop_maddison,...,valoilres_binarize,valoilres_public_diff,valoilres_public_binarize,oilpop_diff,oilpop_binarize,valoilres_impute_diff,valoilres_impute_binarize,oilpop_impute_diff,oilpop_impute_binarize,milexp_pergdpSIPRI_diff
0,1929.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1931.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1932.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1933.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17915,2004.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,10962.026367,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17916,2005.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,11115.380859,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17917,2006.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,11288.252930,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17918,2007.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,11477.447266,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [8]:
oil_df.columns

Index(['year', 'numcode', 'oilreserves_full', 'oilreserves',
       'oilreserves_public', 'newdiscovery_aspo', 'aspo', 'wildcat',
       'endowment', 'pop_maddison', 'ecgrowth', 'efrac', 'lfrac', 'rfrac',
       'incidence2COW', 'onset2COWCS', 'incidenceU', 'onsetUCS', 'Fearon_war',
       'onset_FearonCS', 'Sambanis_war', 'onset_SambanisCS', 'coup',
       'any_coup', 'periregular', 'no_transition', 'milexp_pergdpSIPRI',
       'mountain', 'religion_fractionalization', 'ethnic_fractionalization',
       'language_fractionalization', 'ETHPOL_reynal', 'ETHFRAC_reynal',
       'RELPOL_reynal', 'RELFRAC_reynal', 'leg_british', 'rule_law96',
       'land_area', 'crude1990P', 'smallregion', 'largeregion', 'code3',
       'out_regdisaster', 'oilpop', 'zero_res', 'logoilres', 'oilpop_public',
       'logoilres_public', 'logoilres_full', 'valoilres', 'valoilres_public',
       'logvaloilres', 'logvaloilres_public', 'logvaloilres_full',
       'oilpop_impute', 'logoilres_impute', 'valoilres_imp

In [9]:
# ATT and ATE AIPTW
def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    Double ML estimator for the ATT
    This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
    Return: aiptw of ATE and its standard error
    """
    
    # number of observations
    n = Y.shape[0]
    
    # estimate marginal probability of treatment
    if prob_t is None:
        prob_t = A.mean() 
    
    # att aiptw
    tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
    # influence curve and standard error of aiptw
    phi = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
    std_hat = np.std(phi) / np.sqrt(n)

    return tau_hat, std_hat

def ate_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    Double ML estimator for the ATE
    Return: aiptw of ATE and its standard error
    """
    # number of observations
    n = Y.shape[0]
    
    # ate aiptw
    tau_hat = (Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g)).mean()
  
    # influence curve and standard error of aiptw
    phi = Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g) - tau_hat   
    std_hat = np.std(phi) / np.sqrt(n)

    return tau_hat, std_hat

In [10]:
# Conditional outcome models (Q models)
def make_linear_Q_model():
    ''' A function that returns a linear q model for later use in k-folding'''
    return LinearRegression()

def make_Q_model(output_type:str):
    ''' A function that returns a general ML q model for later use in k-folding'''
    if output_type == 'binary':
        return RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
    return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
# One example: RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)

In [11]:
# Propensity score models (g models)
def make_g_model():
    ''' A function that returns a g model for computing propensity scores'''
    return RandomForestClassifier(n_estimators=100, max_depth=5)
# One example: RandomForestClassifier(n_estimators=100, max_depth=5)

In [12]:
# Functions for K-fold cross-fitting
def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    '''
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    '''

    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
        X_train = X.loc[train_index]
        A_train = A.loc[train_index]
        g = make_model()
        g.fit(X_train, A_train)

        # get predictions for split
        predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]
    
    # sanity check that overlap holds
    assert np.isnan(predictions).sum() == 0
    return predictions

def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    '''
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"
    '''

    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
        X_train = X_w_treatment.loc[train_index]
        y_train = y.loc[train_index]
        q = make_model(output_type)
        q.fit(X_train, y_train)

        if output_type =='binary':
            predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
            predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
        elif output_type == 'continuous':
            predictions0[test_index] = q.predict(X0.loc[test_index])
            predictions1[test_index] = q.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [13]:
model_vars = ['onset2COWCS',
              'valoilres_binarize',
              'ecgrowth',
              'pop_maddison_diff',
              'popdens_diff',
              'democracy_diff',
              'logmountain',
              'ethnic_fractionalization',
              'religion_fractionalization',
              'language_fractionalization',
              'leg_british',
              'numcode',
              'year']

In [14]:
outcome = 'onset2COWCS'
treatment = 'valoilres_binarize'
confounders = [x for x in model_vars if x not in (outcome + treatment)]

In [50]:
def fit_and_run_model(df, outcome:str, treatment:str, confounders:list, make_g_model,
                      make_Q_model, n_splits=5, output_type='binary', ate=True):
    '''
    Function that creates a g, q, and aiptw model based on the 
    given inputs
    
    Inputs: df (pandas df) - the dataframe the variables are contained in
            outcome (str) - the outcome variable
            treatment (str) - the treatment variable
            confounders (lst) - a list of the confounding variables
            make_g_model - the make_g_model function
            make_Q_model - the make_Q_model function
            n_splits (int) - number of splits for the model
            output_type (str) - the desired output type, either binary or continous
            ate (bool) - whether to use ate or alternative att
    
    Returns: tau_hat - the tau hat estimator for the average treatment effect
             std of tau_hat - the standard deviation for the tau_hat estimator
    '''
    df = df.replace({outcome: .00001}, 0)
    df = df[[outcome] + confounders + [treatment]]
    df = df.dropna().reset_index()
    print('Running models for treatment {} and outcome {} on {} samples'.format(treatment, outcome, len(df)))
    n = len(df)
    outcome = df[outcome]
    confounders = df[confounders]
    treatment = df[treatment]
    g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=n_splits)
    
    if min(g) < .01:
        print('\nWARNING:\n Some propensity scores are very small,\n which could '
              'lead to an inflated AIPTW.\n Minimum score = ', min(g))
    if max(g) > .99:
        print('\nWARNING:\n Some propensity scores are very large,\n which could '
              'lead to an inflated AIPTW.\n Maximum score = ', max(g))
    print('G Model has been fit')

    Q0_ml, Q1_ml = outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, \
                                                  n_splits=n_splits, output_type=output_type)
    
    print('Q model has been fit')
    data_and_nuisance_estimates_ml = pd.DataFrame({'g': g, 'Q0': Q0_ml, 'Q1': Q1_ml, 'A': treatment, 'Y': outcome})
    
    # ate aiptw
    if ate:
        tau_hat, std_hat = ate_aiptw(**data_and_nuisance_estimates_ml)
    else: 
        tau_hat, std_hat = att_aiptw(**data_and_nuisance_estimates_ml)
    test_stat = tau_hat / std_hat
    p_value = stats.t.sf(abs(test_stat), df=(n-1))
    print('AIPTW model has been fit. Returning \u03C4 hat and its standard deviation')
    print('\u03C4 hat = {}, std = {}, test statistic = {}, p-value = {}\n'.format(round(tau_hat, 5), round(std_hat, 5), round(test_stat, 5), round(p_value, 5)))
    return tau_hat, std_hat, test_stat, p_value

In [32]:
treatment_lst = ['valoilres_binarize', # value of oil reserves
               'valoilres_public_binarize', # value of oil reserves from public data
               'oilpop_binarize', # oil reserves per capita in million barrels per 1000 persons
               'valoilres_impute_binarize', # value of oilpop_impute (multiply by crude oil price)
               'oilpop_impute_binarize']

outcome_lst = ['onset2COWCS',
               'onsetUCS',
               'coup',
               'periregular',
               'milexp_pergdpSIPRI_diff']

In [33]:
undemocratic_countries_df = oil_df[oil_df['democracy'] <= oil_df['democracy'].median()]
democratic_countries_df = oil_df[oil_df['democracy'] > oil_df['democracy'].median()]

In [34]:
df_dict = {'model': [], 'treatment': [], 'outcome': [], 'tau_hat': [], 'tau_std': [], 'test_stat': [], 'p_value': []}
for treat in treatment_lst:
    for out in outcome_lst:
        if len(oil_df[out].value_counts()) == 2:
            output = 'binary'
        else:
            output = 'continuous'
        tau_hat, tau_std, test_stat, p_value = fit_and_run_model(oil_df, out, treat, confounders, make_g_model, make_Q_model, output_type=output)
        df_dict['model'].append('ate')
        df_dict['treatment'].append(treat)
        df_dict['outcome'].append(out)
        df_dict['tau_hat'].append(tau_hat)
        df_dict['tau_std'].append(tau_std)
        df_dict['test_stat'].append(test_stat)
        df_dict['p_value'].append(p_value)

ate_results_df = pd.DataFrame(data=df_dict)

Running models for treatment valoilres_binarize and outcome onset2COWCS on 5174 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00408, std = 0.00447, test statistic = 0.91161, p-value = 0.18101

Running models for treatment valoilres_binarize and outcome onsetUCS on 4754 samples

 Some propensity scores are very small,
 which could lead to an inflated AIPTW.
 Minimum score =  0.008416306083857506
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00963, std = 0.0077, test statistic = 1.25062, p-value = 0.10557

Running models for treatment valoilres_binarize and outcome coup on 5364 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = -0.00395, std = 0.0114, test statistic = -0.34647, p-value = 0.3645

Running models for treatment valoilres_binarize and outcome per

In [35]:
df_dict = {'model': [], 'treatment': [], 'outcome': [], 'tau_hat': [], 'tau_std': [], 'test_stat': [], 'p_value': []}
for treat in treatment_lst:
    for out in outcome_lst:
        if len(oil_df[out].value_counts()) == 2:
            output = 'binary'
        else:
            output = 'continuous'
        tau_hat, tau_std, test_stat, p_value = fit_and_run_model(oil_df, out, treat, confounders, make_g_model, make_Q_model, output_type=output, ate=False)
        df_dict['model'].append('att')
        df_dict['treatment'].append(treat)
        df_dict['outcome'].append(out)
        df_dict['tau_hat'].append(tau_hat)
        df_dict['tau_std'].append(tau_std)
        df_dict['test_stat'].append(test_stat)
        df_dict['p_value'].append(p_value)

att_results_df = pd.DataFrame(data=df_dict)

Running models for treatment valoilres_binarize and outcome onset2COWCS on 5174 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00282, std = 0.00497, test statistic = 0.56752, p-value = 0.28519

Running models for treatment valoilres_binarize and outcome onsetUCS on 4754 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00654, std = 0.00866, test statistic = 0.75555, p-value = 0.22498

Running models for treatment valoilres_binarize and outcome coup on 5364 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = -0.01097, std = 0.01316, test statistic = -0.83337, p-value = 0.20234

Running models for treatment valoilres_binarize and outcome periregular on 3312 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its 

In [36]:
results_df = pd.concat([ate_results_df, att_results_df])
results_df

,model,treatment,outcome,tau_hat,tau_std,test_stat,p_value
0,ate,valoilres_binarize,onset2COWCS,0.004077,0.004472,0.911614,0.181007
1,ate,valoilres_binarize,onsetUCS,0.009631,0.007701,1.250619,0.105568
2,ate,valoilres_binarize,coup,-0.003951,0.011404,-0.346466,0.364503
3,ate,valoilres_binarize,periregular,-0.012738,0.004746,-2.684003,0.003655
4,ate,valoilres_binarize,milexp_pergdpSIPRI_diff,0.502530,0.424950,1.182563,0.118599
5,ate,valoilres_public_binarize,onset2COWCS,0.002791,0.004351,0.641327,0.260671
6,ate,valoilres_public_binarize,onsetUCS,0.017497,0.006702,2.610869,0.004530
7,ate,valoilres_public_binarize,coup,0.023245,0.011498,2.021558,0.021635
8,ate,valoilres_public_binarize,periregular,-0.006905,0.006585,-1.048552,0.147241
9,ate,valoilres_public_binarize,milexp_pergdpSIPRI_diff,0.339979,0.257640,1.319591,0.093585


In [37]:
bonferonni = .05 / len(results_df)

In [38]:
results_df[results_df['p_value'] < bonferonni]

,model,treatment,outcome,tau_hat,tau_std,test_stat,p_value


In [56]:
df_dict = {'model': [], 'democracy': [], 'treatment': [], 'outcome': [], 'tau_hat': [], 'tau_std': [], 'test_stat': [], 'p_value': []}
for treat in treatment_lst:
    for ate_bool in [True, False]:
        for democracy in [True, False]:
            if democracy:
                df = oil_df[oil_df['democracy'] > .005]
            else:
                df = oil_df[oil_df['democracy'] < .005]
            tau_hat, tau_std, test_stat, p_value = fit_and_run_model(df, 'milexp_pergdpSIPRI_diff', treat, confounders, make_g_model, make_Q_model, output_type='continuous', ate=False)
            if ate_bool:
                df_dict['model'].append('ate')
            else:
                df_dict['model'].append('att')
            df_dict['treatment'].append(treat)
            df_dict['outcome'].append(out)
            df_dict['tau_hat'].append(tau_hat)
            df_dict['tau_std'].append(tau_std)
            df_dict['democracy'].append(democracy)
            df_dict['test_stat'].append(test_stat)
            df_dict['p_value'].append(p_value)

defense_burden_df = pd.DataFrame(data=df_dict)

Running models for treatment valoilres_binarize and outcome milexp_pergdpSIPRI_diff on 809 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00527, std = 0.0302, test statistic = 0.17436, p-value = 0.43081

Running models for treatment valoilres_binarize and outcome milexp_pergdpSIPRI_diff on 459 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 1.52574, std = 1.30218, test statistic = 1.17168, p-value = 0.12097

Running models for treatment valoilres_binarize and outcome milexp_pergdpSIPRI_diff on 809 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00426, std = 0.02985, test statistic = 0.14259, p-value = 0.44333

Running models for treatment valoilres_binarize and outcome milexp_pergdpSIPRI_diff on 459 samples
G Model has been fit
Q model has been fit

In [57]:
defense_bonferonni = .05 / len(defense_burden_df)

In [58]:
defense_burden_df

,model,democracy,treatment,outcome,tau_hat,tau_std,test_stat,p_value
0,ate,True,valoilres_binarize,milexp_pergdpSIPRI_diff,0.005265,0.030197,0.174365,0.430811
1,ate,False,valoilres_binarize,milexp_pergdpSIPRI_diff,1.525741,1.302178,1.171683,0.120967
2,att,True,valoilres_binarize,milexp_pergdpSIPRI_diff,0.004256,0.029852,0.142587,0.443326
3,att,False,valoilres_binarize,milexp_pergdpSIPRI_diff,1.387649,1.249108,1.110912,0.133595
4,ate,True,valoilres_public_binarize,milexp_pergdpSIPRI_diff,-0.005334,0.028881,-0.184706,0.426749
5,ate,False,valoilres_public_binarize,milexp_pergdpSIPRI_diff,1.557181,1.058990,1.470440,0.071027
6,att,True,valoilres_public_binarize,milexp_pergdpSIPRI_diff,-0.007056,0.028785,-0.245122,0.403207
7,att,False,valoilres_public_binarize,milexp_pergdpSIPRI_diff,1.366253,0.988787,1.381746,0.083824
8,ate,True,oilpop_binarize,milexp_pergdpSIPRI_diff,0.087243,0.091937,0.948951,0.171465
9,ate,False,oilpop_binarize,milexp_pergdpSIPRI_diff,7.458193,6.699602,1.113229,0.133097


In [42]:
aspo_df = pd.read_stata('../Data/data/aspo_full_matching.dta')

In [43]:
aspo_df

,index,year,numcode,oilreserves_full,oilreserves,oilreserves_public,newdiscovery_aspo,imputedzero,aspo,wildcat,...,l1language_fractionalization,l2language_fractionalization,l5language_fractionalization,l10language_fractionalization,l1leg_british,l2leg_british,l5leg_british,l10leg_british,wildcat_diff_binary,success
0,0,1929-01-01,8,0.281900,0.281900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1930-01-01,8,0.282660,0.282660,NaN,0.00076,NaN,1.0,1.00000,...,0.000399,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1.0
2,2,1931-01-01,8,0.282660,0.282660,NaN,0.00001,NaN,1.0,0.00001,...,0.000399,0.000399,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0
3,3,1932-01-01,8,0.282660,0.282660,NaN,0.00001,NaN,1.0,0.00001,...,0.000399,0.000399,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0
4,4,1933-01-01,8,0.282649,0.282649,NaN,0.00001,NaN,1.0,0.00001,...,0.000399,0.000399,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5063,5115,2004-01-01,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
5064,5116,2005-01-01,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
5065,5117,2006-01-01,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
5066,5118,2007-01-01,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [51]:
confounders = ['ecgrowth',
 'popdens_diff',
 'democracy_diff',
 'logmountain',
 'ethnic_fractionalization',
 'religion_fractionalization',
 'language_fractionalization',
 'leg_british']

In [52]:
fit_and_run_model(aspo_df, 'onset2COWCS', 'wildcat_diff_binary', confounders, make_g_model, make_Q_model, output_type=output, ate=True)

Running models for treatment wildcat_diff_binary and outcome onset2COWCS on 2791 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = -0.00241, std = 0.00524, test statistic = -0.45936, p-value = 0.32301



(-0.0024053897169974375,
 0.005236383737259125,
 -0.45936085621113937,
 0.3230054203350724)